In [1]:
import numpy as np
import matplotlib.pyplot as plt
import leidenalg as la
import igraph as ig

In [6]:
class temporal_network:#object for creating node-aligned(every node exists every layer)
                       #with diagonal coupling(inter-layer edges exist only between node to itself)
        
    ##################################
    # TODO: extend omega(scalar) for vector and matrix
    ##################################
        
    def __init__(self, size, length, data, **kwargs):
        
        if length < 1: return('Object should be a multilayer network with at least 2 layers')
        if size < 3: return('Layers must have at least 3 nodes')
        
        self.size = size # number of nodes in every layer
        self.length = length # number of layers
        self.nodes = [i for i in range(self.size)]
        
        #### data: supra__adjacency, list_adjacency, edge_list
        
        ##         if supra__adjacency: creates the list adjacency matrix
        ##
        ##                     --- additional arguments ---
        ##                       - supra_adjacency: supra adjacency matrix of shape (size*time x size*time)
        
        
        ##
        ##         if edge__list: creates directed weighted multilayer network from the egde quadraplets
        ##                      given of the form (i,j,w,t). supra_adjacency and list_adjacency matrices 
        ##                      are automatically created. 
        ##
        ##                     --- additional arguments ---
        ##                       - edge_list: list of quadreplets e.g. [(0,2,w1,1),(2,1,w2,1),(0,1,w3,2),(0,2,w4,2)]
        ##                       - omega: interlayer coupling strength, can be a float only for now
        ##                       - kind: if ordinal, only adjacent layers gets connected with strength scalar 'omega'
        ##                               if cardinal, all layers get connected w/ each other w/ strength scalar 'omega'
        
        
        ##         if list__adjacency: creates the supra adjacency matrix from given list of adjacency matrices
        ##                             of monolayer networks
        ##                             TODO:add a warning to check if the adjacency matrices are node-aligned
        ##                      
        ##              
        ##                     --- additional arguments ---
        ##                       - list_adjacency: list of length 'length' that contains individual adjacency
        ##                                         matrices of each layer that are numpy arrays
        ##                       - omega: interlayer coupling strength, can be a float only for now
        ##                       - kind : if ordinal, only adjacent layers gets connected w/ strength scalar'omega'
        ##                                if cardinal, all layers get connected w/ each other w/ strength scalar'omega'
        ##
        ####
                    
        if  data == 'supra__adjacency':
            self.supra_adjacency = kwargs['supra_adjacency']
            list_adjacency = [ [] for i in range(length) ]
            
            for i in range(self.length):
                list_adjacency[i] = self.supra_adjacency[i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size]
            
            self.list_adjacency = list_adjacency
            
            edge_list = []
            for i in range(self.length):
                A = self.list_adjacency[i]
                firing = np.transpose(np.nonzero(A))
                for j,m in enumerate(firing):
                    quadreplet =(m[0],m[1],A[m[0],m[1]],i)
                    edge_list.append(quadreplet)
            self.edgelist = edge_list
                
        
        elif data == 'edge__list':
            self.edgelist = kwargs['edge_list']
            supra_adjacency = np.zeros((self.size*self.length,self.size*self.length))
            list_adjacency = [ [] for i in range(self.length) ]
            for q in range(self.length):
                list_adjacency[q]=np.zeros((self.size,self.size))
            
            for k,e in enumerate(self.edgelist):
                i,j,w,t = e[0], e[1], e[2],e[3]
                supra_adjacency[self.size*(t)+i][self.size*(t)+j] = w
                list_adjacency[t][i][j] = w

        
            ##filling off-diagonal blocks
            if kwargs['kind'] == 'ordinal':
                for n in range(self.size*(self.length-1)):
                    supra_adjacency[n][n+self.size] = kwargs['omega']
                    supra_adjacency[n+self.size][n] = kwargs['omega']
                
            elif kwargs['kind'] == 'cardinal':
                i = 0
                while self.length-i != 0:
                    i = i+1
                    for n in range(self.size*(self.length-i)):
                        supra_adjacency[n][n+i*self.size] = kwargs['omega']
                        supra_adjacency[n+i*self.size][n] = kwargs['omega']
            
            self.supra_adjacency = supra_adjacency
            self.list_adjacency = list_adjacency
            
        elif data == 'list__adjacency':
            self.list_adjacency = kwargs['list_adjacency']
            supra_adjacency = np.zeros((self.size*self.length,self.size*self.length))
            
            for i in range(self.length):
                supra_adjacency[i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size] = self.list_adjacency[i]
            
            ##filling off-diagonal blocks
            if kwargs['kind'] == 'ordinal':
                for n in range(self.size*(self.length-1)):
                    supra_adjacency[n][n+self.size] = kwargs['omega']
                    supra_adjacency[n+self.size][n] = kwargs['omega']
                
            elif kwargs['kind'] == 'cardinal':
                i = 0
                while self.length-i != 0:
                    i = i+1
                    for n in range(self.size*(self.length-i)):
                        supra_adjacency[n][n+i*self.size] = kwargs['omega']
                        supra_adjacency[n+i*self.size][n] = kwargs['omega']
            
            self.supra_adjacency = supra_adjacency
            
            edge_list = []
            for i in range(self.length):
                A = self.list_adjacency[i]
                firing = np.transpose(np.nonzero(A))
                for j,m in enumerate(firing):
                    quadreplet =(m[0],m[1],A[m[0],m[1]],i)
                    edge_list.append(quadreplet)
            self.edgelist = edge_list
            
    def aggragate(self, normalized = True):
        t = self.length
        n = self.size
        aggragated = np.zeros((n,n))
        
        for i,c in enumerate(self.list_adjacency):
            aggragated = aggragated + c
            
        if normalized: return (aggragated/t)
        else: return (aggragated)
            
    def modularity_matrix(self, omega, gamma):##TODO: fix modularity matrix
        N = self.size
        T = self.length
        B = np.zeros((N*T,N*T))
        two_mu = 0
        for i in range(T):
            k = np.sum(self.multi_array[i],0)
            two_m = np.sum(k,0)
            two_mu = two_mu + two_m
            B[i*N:(i+1)*N,i*N:(i+1)*N] = self.multi_array[i] - (gamma * k.T*k)/(two_m)
        two_mu = two_mu + 2*omega*N*(T-1)
        
        for p in range(N*(T-1)):
            B[p][p+N] = omega 
            B[p+N][p] = omega
            
        return(B)
    
    def edgelist2edges(self):
        T = self.length
        all_edges = [[] for i in range(T)]
        weights = []
        dtype = [('row',int),('column',int),('weight',float),('layer',int)]
        for k,e in enumerate(np.sort(np.array(self.edgelist, dtype=dtype),order='layer')):
            i,j,w,t = e[0], e[1], e[2],e[3]
            pair = (i,j)
            all_edges[t].append(pair)
            weights.append(w)
        return (all_edges, weights)
    
    def graph_formula(self):
        formula = '1'
        for i in range(2,self.length + 1):
            temp = formula
            formula = temp + ' -- %d'%i
        return(formula)
    
    def create_igraph(self, omega):
        T = self.length
        N = self.size
        G = []
        edges = self.edgelist2edges()[0]
        weights = self.edgelist2edges()[1]
        for i in range(T):
            G.append(ig.Graph())
            G[i].add_vertices(N)
            G[i].add_edges(edges[i])
            G[i].es['weight'] = weights
            G[i].vs['id'] = list(range(N))
            G[i].vs['node_size'] = 0
            
        G_coupling = ig.Graph.Formula(self.graph_formula());
        G_coupling.es['weight'] = omega; # Interslice coupling strength
        G_coupling.vs['slice'] = G
        
        layers, interslice_layer, G_full = la.time_slices_to_layers(G,interslice_weight=omega);
        partitions = [la.CPMVertexPartition(H, node_sizes='node_size',
                                            weights='weight', resolution_parameter=omega) for H in layers];
        interslice_partition = la.CPMVertexPartition(interslice_layer, resolution_parameter=omega)
                                                     
        return(partitions, interslice_partition)
    
    def leiden(self):
        optimiser = la.Optimiser()
        diff = optimiser.optimise_partition_multiplex(self.create_igraph(omega=0)[0] + [self.create_igraph(omega=0)[1]])

        return(diff)

In [7]:
supra = np.array(([[.2,0,.1,1,0,0],[.1,0,.3,0,1,0],[.2,.5,.9,0,0,1],[1,0,0,.1,0,.6],[0,1,0,.1,.2,.8],[0,0,1,.1,.2,.3]]))
edges = [(0,0,.2,0),(0,2,.1,0),(1,0,.1,0),(1,2,.3,0),(2,0,.2,0),(2,1,.5,0),(2,2,.9,0),(0,0,.1,1),(0,2,.6,1),(1,0,.1,1),(1,1,.2,1),(1,2,.8,1),(2,0,.1,1),(2,1,.2,1),(2,2,.3,1)]
adjs = [np.array([[.2,0,.1],[.1,0,.3],[.2,.5,.9]]),np.array([[.1,0,.6],[.1,.2,.8],[.1,.2,.3]])]
test_supra = temporal_network(3,2,data = 'supra__adjacency',supra_adjacency=supra,omega=1,kind='ordinal')
test_edge = temporal_network(3,2,data = 'edge__list',edge_list=edges,omega=1,kind='ordinal')
test_list = temporal_network(3,2,data = 'list__adjacency',list_adjacency=adjs,omega=1,kind='ordinal')

In [4]:
#test_supra.edgelist,test_supra.list_adjacency,test_supra.supra_adjacency
#test_edge.edgelist,test_edge.list_adjacency,test_edge.supra_adjacency
#test_list.edgelist,test_list.list_adjacency,test_list.supra_adjacency

In [8]:
diff_try = test_supra.leiden()
part, int_part = test_supra.create_igraph(omega = 0.1)
print(int_part,part)

Clustering with 6 elements and 6 clusters
[0] 0
[1] 1
[2] 2
[3] 3
[4] 4
[5] 5 [<leidenalg.VertexPartition.CPMVertexPartition object at 0x7fa957d31e90>, <leidenalg.VertexPartition.CPMVertexPartition object at 0x7fa957d31f10>]
